## Introduction

A machine learning algorithm is implemented in order to predict whether coursera users will continue their subscription. The dataset is a sample of subscriptions that were iniciated in 2021, all snapshotted at a particular date before the subscription was cancelled.


Regardless the reason, Coursera has a vested interest in understanding the likelihood of each individual learner to retain in their subscription so that resources can be allocated appropriately to support learners across the various stages of their learning journeys.

## Understanding the Datasets

### Train vs. Test
In this competition, you’ll gain access to two datasets that are samples of past specialization subscriptions that contain information about the learner, the specialization, and the learner's activity in the subscription thus far. One dataset is titled `train.csv` and the other is titled `test.csv`.

`train.csv` contains 70% of the overall sample (509,837 subscriptions to be exact) and importantly, will reveal whether or not the subscription was continued into the next month (the “ground truth”).

The `test.csv` dataset contains the exact same information about the remaining segment of the overall sample (217,921 subscriptions to be exact), but does not disclose the “ground truth” for each subscription. It’s your job to predict this outcome!

Using the patterns you find in the `train.csv` data, predict whether the subscriptions in `test.csv` will be continued for another month, or not.

### Dataset descriptions
Both `train.csv` and `test.csv` contain one row for each unique specialization subscription. For each subscription, a single observation (`subscription_id`) is included as of a particular date (`observation_dt`) during which the subscription was active. This date was chosen at random from all the dates during which the subscription was active. In some instances it is soon after the subscription was initiated; in other instances, it is several months after the subscription was initiated and after several previous payments were made. Therefore, your model will have to be able to adapt to different stages of the subscription.

In addition to those identifier columns, the `train.csv` dataset also contains the target label for the task, a binary column `is_retained`.

Besides that column, both datasets have an identical set of features that can be used to train the model to make predictions. Descriptions of each feature are shown below. 

In [2]:
import pandas as pd
data_descriptions = pd.read_csv('data_descriptions.csv')
pd.set_option('display.max_colwidth', None)
data_descriptions

,Column_name,Column_type,Data_type,Description
0,subscription_id,Identifier,character,Unique identifier of each subscription
1,observation_dt,Identifier,date,The date on which the subscription was observed to calculate the features in the dataset. It was chosen at random amongst all the dates between the start of the subscription and the end of the subscription (before cancellation)
2,is_retained,Target,Integer,"TRAINING SET ONLY! 0 = the learner cancelled their subscription before next payment, 1 = the learner made an additional payment in this subscription"
3,specialization_id,Feature - Specialization Info,character,Unique identifier of a specialization (each subscription gives a learner access to a particular specialization)
4,cnt_courses_in_specialization,Feature - Specialization Info,integer,number of courses in the specialization
5,specialization_domain,Feature - Specialization Info,character,"primary domain of the specialization (Computer Science, Data Science, etc.)"
6,is_professional_certificate,Feature - Specialization Info,boolean,"BOOLEAN for whether the specialization is a ""professional certicate"" (a special type of specialization that awards completers with an industry-sponsored credential)"
7,is_gateway_certificate,Feature - Specialization Info,boolean,"BOOLEAN for whether the specialization is a ""gateway certificate"" (a special type of specialization geared towards learners starting in a new field)"
8,learner_days_since_registration,Feature - Learner Info,integer,Days from coursera registration date to the date on which the observation is made
9,learner_country_group,Feature - Learner Info,character,"the region of the world that the learner is from (United States, East Asia, etc.)"


In [3]:
# Import required packages

# Data packages
import pandas as pd
import numpy as np
import tensorflow as tf

# Machine Learning / Classification packages
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

# Visualization Packages
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

2022-12-27 18:15:09.560769: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Import any other packages you may want to use
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.activations import linear, relu, sigmoid


## Load the Data

Let's start by loading the dataset `train.csv` into a dataframe `train_df`, and `test.csv` into a dataframe `test_df` and display the shape of the dataframes.

In [30]:
train_df = pd.read_csv("train.csv")
train_df.shape

/var/folders/sc/c663j6bx65391kdjgk7jy_d00000gn/T/ipykernel_91916/3935830336.py:1: DtypeWarning: Columns (6,7,25,31) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("train.csv")


(413955, 37)

In [31]:
# test_df = pd.read_csv("test.csv")
# test_df.shape

In [32]:
train_df.head()

,subscription_id,observation_dt,is_retained,specialization_id,cnt_courses_in_specialization,specialization_domain,is_professional_certificate,is_gateway_certificate,learner_days_since_registration,learner_country_group,...,cnt_enrollments_completed_during_payment_period,cnt_enrollments_active_during_payment_period,cnt_items_completed_during_payment_period,cnt_graded_items_completed_during_payment_period,is_active_capstone_during_pay_period,sum_hours_learning_before_payment_period,sum_hours_learning_during_payment_period,cnt_days_active_before_payment_period,cnt_days_active_during_payment_period,cnt_days_since_last_activity
0,--rKikbGEeyQHQqIvaM5IQ,2022-05-04,1.0,kr43OcbTEeqeNBKhfgCLyw,8.0,Data Science,True,True,2321.0,Northern Europe,...,0.0,0.0,0.0,0.0,False,73.783333,0.000000,68.0,0.0,20.0
1,-0XGzEq2EeyimBISGRuNeQ,2021-11-30,0.0,Q0Fc_Yl0EeqdTApgQ4tM7Q,6.0,Data Science,True,False,612.0,Northern Europe,...,0.0,0.0,0.0,0.0,False,0.850000,0.000000,7.0,2.0,0.0
2,-1P9kOb6EeuRugq1Liq62w,2021-08-13,0.0,9kmimrDIEeqxzQqieMm42w,6.0,Business,True,True,27.0,Australia and New Zealand,...,0.0,1.0,12.0,2.0,False,1.833333,2.983333,2.0,1.0,18.0
3,-2ifTJZbEeuIuRKpAhovaw,2021-08-03,1.0,7lHCSlFIEeeffRIHljDI_g,5.0,Information Technology,True,True,120.0,United States,...,0.0,2.0,83.0,9.0,False,18.450000,7.100000,18.0,4.0,3.0
4,-5YKZbchEeufeAq6C_fAOw,2021-06-04,0.0,kr43OcbTEeqeNBKhfgCLyw,8.0,Data Science,True,True,1228.0,India,...,1.0,1.0,61.0,4.0,False,29.566667,15.250000,18.0,8.0,1.0


In [33]:
# Get 60% of the dataset as the training set. Put the remaining 40% in a temporary data frame: temp
# Split the 40% subset above into two: one half for cross validation and the other for the test set
train, temp = train_test_split(train_df, test_size=0.4, random_state = 1234)
validation, test = train_test_split(temp, test_size = 0.5, random_state = 1234)

del temp


In [34]:
train.shape

(248373, 37)

In [35]:
validation.shape

(82791, 37)

In [36]:
test.shape

(82791, 37)

### preprocessing training set

In [10]:

def pre_processing(path, frac_features, frac_features_names, drop_features):
#     drop_features: list of strings with the names of the categorical features 
#     binary_features: list of string witht the names of the binary columns
#     target_col: string

    df = pd.read_csv(path)
    df = df.dropna()
    
#     y_train = df[target_col].values

    total_courses_count = "cnt_courses_in_specialization"
    
    for feature, name in zip(frac_features, frac_features_names):
        
        df[name] = df[feature] / df[total_courses_count]

    df = df.drop(columns = drop_features)
    
    return df
    
def scaling(df, binary_features, is_train): 
    
# scaling the data using the standard scaler
    binary_cols = df[binary_features].values
    binary_cols = binary_cols.astype(int)
    cols_to_scale = list(set(df.columns).difference(set(binary_features)))
    
    if is_train:
    
        cols_scaled = scaler.fit_transform(df[cols_to_scale])
        
    else:
        
        cols_scaled = scaler.transform(df[cols_to_scale])
        
    scaled_df = np.concatenate((binary_cols, cols_scaled), axis = 1)
    
    return scaled_df


    

In [11]:
# convert integers of course enrollment to fractions
frac_features = ["cnt_enrollments_started_before_payment_period", 
                                    "cnt_enrollments_completed_before_payment_period", 
                                    "cnt_enrollments_active_before_payment_period", 
                                    "cnt_enrollments_started_during_payment_period", 
                                    "cnt_enrollments_completed_during_payment_period", 
                                    "cnt_enrollments_active_during_payment_period",]

frac_features_names = ["frc_enrollments_started_before_payment_period", 
                       "frc_enrollments_completed_before_payment_period", 
                       "frc_enrollments_active_before_payment_period", 
                       "frc_enrollments_started_during_payment_period", 
                       "frc_enrollments_completed_during_payment_period", 
                       "frc_enrollments_active_during_payment_period"]

# drop the categorical features
drop_features = ["specialization_id", "specialization_domain", "learner_country_group", 
                                    "learner_gender", "observation_dt", "subscription_id", 
                                    "learner_cnt_other_transactions_past", 
                                    "cnt_enrollments_started_before_payment_period", 
                                    "cnt_enrollments_completed_before_payment_period", 
                                    "cnt_enrollments_active_before_payment_period", 
                                    "cnt_enrollments_started_during_payment_period", 
                                    "cnt_enrollments_completed_during_payment_period", 
                                    "cnt_enrollments_active_during_payment_period", 
                                    "cnt_courses_in_specialization",]

binary_features = ["is_professional_certificate", "is_gateway_certificate", 
                   "is_subscription_started_with_free_trial", "is_active_capstone_during_pay_period",]

target_col = ["is_retained"]

In [12]:
# preprocessing training set
path = "train.csv"
data = pre_processing(path, frac_features, frac_features_names, drop_features)

y_train = data.is_retained
data = data.drop(columns = y_train.name)

# scaling training data
scaler = StandardScaler()
X_train = scaling(data, binary_features, 1 == 1)

/var/folders/sc/c663j6bx65391kdjgk7jy_d00000gn/T/ipykernel_91916/3170332484.py:6: DtypeWarning: Columns (6,7,25,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [ ]:
# preprocessing test set
path_test = "test.csv"
test_ids = test_df[['subscription_id']]
test_data = pre_processing(path_test, frac_features, frac_features_names, drop_features)

y_test = test_data.is_retained
test_data = test_data.drop(columns = y_test.name)

# scaling test data
X_test = scaling(test_data, binary_features, 1 == 0)



In [13]:
data.head()

,is_professional_certificate,is_gateway_certificate,learner_days_since_registration,learner_cnt_other_courses_active,learner_cnt_other_courses_paid_active,learner_cnt_other_courses_items_completed,learner_cnt_other_courses_paid_items_completed,learner_other_revenue,subscription_period_order,days_since_last_payment,...,sum_hours_learning_during_payment_period,cnt_days_active_before_payment_period,cnt_days_active_during_payment_period,cnt_days_since_last_activity,frc_enrollments_started_before_payment_period,frc_enrollments_completed_before_payment_period,frc_enrollments_active_before_payment_period,frc_enrollments_started_during_payment_period,frc_enrollments_completed_during_payment_period,frc_enrollments_active_during_payment_period
0,True,True,2321.0,8.0,0.0,88.0,0.0,0.00,6.0,18.0,...,0.000000,68.0,0.0,20.0,0.625000,0.500,0.625000,0.000,0.000,0.000000
1,True,False,612.0,52.0,2.0,209.0,75.0,49.41,1.0,1.0,...,0.000000,7.0,2.0,0.0,0.166667,0.000,0.166667,0.000,0.000,0.000000
2,True,True,27.0,5.0,0.0,5.0,0.0,0.00,1.0,18.0,...,2.983333,2.0,1.0,18.0,0.166667,0.000,0.166667,0.000,0.000,0.166667
3,True,True,120.0,0.0,0.0,0.0,0.0,0.00,4.0,21.0,...,7.100000,18.0,4.0,3.0,0.400000,0.200,0.400000,0.200,0.000,0.400000
4,True,True,1228.0,0.0,0.0,0.0,0.0,0.00,1.0,10.0,...,15.250000,18.0,8.0,1.0,0.375000,0.125,0.375000,0.125,0.125,0.125000


##### preprocessing test set

In [183]:
test_ids = test_df[['subscription_id']]

In [184]:
# clean and scale the test set consistently with the training set

test_df = test_df.dropna()

# convert integers of courses to fractions
test_df["frc_enrollments_started_before_payment_period"] = test_df.cnt_enrollments_started_before_payment_period / test_df.cnt_courses_in_specialization
test_df["frc_enrollments_completed_before_payment_period"] = test_df.cnt_enrollments_completed_before_payment_period / test_df.cnt_courses_in_specialization
test_df["frc_enrollments_active_before_payment_period"] = test_df.cnt_enrollments_active_before_payment_period / test_df.cnt_courses_in_specialization

test_df["frc_enrollments_started_during_payment_period"] = test_df.cnt_enrollments_started_during_payment_period / test_df.cnt_courses_in_specialization
test_df["frc_enrollments_completed_during_payment_period"] = test_df.cnt_enrollments_completed_during_payment_period / test_df.cnt_courses_in_specialization
test_df["frc_enrollments_active_during_payment_period"] = test_df.cnt_enrollments_active_during_payment_period / test_df.cnt_courses_in_specialization



# drop the categorical features
test_df = test_df.drop(columns = ["specialization_id", "specialization_domain", "learner_country_group", 
                                    "learner_gender", "observation_dt", "subscription_id", 
                                  "learner_cnt_other_transactions_past", 
                                  "cnt_enrollments_started_before_payment_period", 
                                    "cnt_enrollments_completed_before_payment_period", 
                                    "cnt_enrollments_active_before_payment_period", 
                                    "cnt_enrollments_started_during_payment_period", 
                                    "cnt_enrollments_completed_during_payment_period", 
                                    "cnt_enrollments_active_during_payment_period", 
                                    "cnt_courses_in_specialization",
                                 ])


In [185]:
# convert the boolean features to integers and use a standard scaler to scale the other numerical features
binary_cols_names = ["is_professional_certificate", "is_gateway_certificate", 
               "is_subscription_started_with_free_trial", "is_active_capstone_during_pay_period"]
binary_cols = test_df[binary_cols_names].values
binary_cols = binary_cols.astype(int)

cols_to_scale = list(set(test_df.columns).difference(set(binary_cols_names)))


cols_scaled = scaler.transform(test_df[cols_to_scale])
X_test = np.concatenate((binary_cols, cols_scaled), axis = 1)


In [186]:
# split data into training and cross validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=123)



In [187]:
X_train.shape

(372557, 28)

In [186]:
# attributes = ["is_retained", "cnt_items_completed_during_payment_period", "learner_days_since_registration", 
#               "sum_hours_learning_before_payment_period", "sum_hours_learning_during_payment_period"]
# scatter_matrix(train_df[attributes], figsize = (10, 10))
# plt.show()

In [184]:
# train_df.plot(kind = "scatter", x = "sum_hours_learning_during_payment_period", 
#               y = "cnt_items_completed_during_payment_period", alpha = 0.1)
# plt.show()

In [111]:
# corr_matrix = train_df.corr()
# corr_matrix["is_retained"].sort_values(ascending = False)

### model

In [188]:
model = Sequential(
    [
        Dense(256, input_shape = (28,), activation = "relu"),
        Dense(128, activation = "relu"),
        Dense(64, activation = "relu"),
        Dense(1, activation = "sigmoid")
    ]
)


In [189]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(0.001),
    metrics = [
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
    ],
)

model.fit(
    X_train,y_train,
    validation_data = (X_val, y_val),
    epochs = 100,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
        monitor = "val_loss",
        mode = "min",
        patience = 2,
        verbose = 0,
        restore_best_weights = True,
    ),
                ]
)

Train on 372557 samples, validate on 41396 samples
Epoch 1/100
372557/372557 [==============================] - 36s 97us/sample - loss: 0.5942 - binary_accuracy: 0.6796 - precision_8: 0.6910 - recall_8: 0.7430 - val_loss: 0.5870 - val_binary_accuracy: 0.6860 - val_precision_8: 0.6896 - val_recall_8: 0.7691
Epoch 2/100
372557/372557 [==============================] - 35s 94us/sample - loss: 0.5882 - binary_accuracy: 0.6853 - precision_8: 0.6966 - recall_8: 0.7461 - val_loss: 0.5860 - val_binary_accuracy: 0.6869 - val_precision_8: 0.6979 - val_recall_8: 0.7490
Epoch 3/100
372557/372557 [==============================] - 35s 94us/sample - loss: 0.5864 - binary_accuracy: 0.6863 - precision_8: 0.6966 - recall_8: 0.7493 - val_loss: 0.5837 - val_binary_accuracy: 0.6879 - val_precision_8: 0.7016 - val_recall_8: 0.7422
Epoch 4/100
372557/372557 [==============================] - 35s 95us/sample - loss: 0.5852 - binary_accuracy: 0.6872 - precision_8: 0.6970 - recall_8: 0.7510 - val_loss: 0.5863 

## Make predictions (required)

Remember you should create a dataframe named `prediction_df` with exactly 217,921 entries plus a header row attempting to predict the likelihood of retention for subscriptions in `test_df`. Your submission will throw an error if you have extra columns (beyond `subscription_id` and `predicted_probaility`) or extra rows.

The file should have exactly 2 columns:
`subscription_id` (sorted in any order)
`predicted_probability` (contains your numeric predicted probabilities between 0 and 1, e.g. from `estimator.predict_proba(X, y)[:, 1]`)

The naming convention of the dataframe and columns are critical for our autograding, so please make sure to use the exact naming conventions of `prediction_df` with column names `subscription_id` and `predicted_probability`!

### Example prediction submission:

The code below is a very naive prediction method that simply predicts retention using a Dummy Classifier. This is used as just an example showing the submission format required. Please change/alter/delete this code below and create your own improved prediction methods for generating `prediction_df`.

**PLEASE CHANGE CODE BELOW TO IMPLEMENT YOUR OWN PREDICTIONS**

In [152]:
predictions = model.predict(X_test).flatten()

In [153]:
# Combine predictions with label column into a dataframe

prediction_df = pd.DataFrame({'subscription_id': test_ids.values[:, 0], 
                             'predicted_probability': predictions})

In [154]:
prediction_df

,subscription_id,predicted_probability
0,-1flsPG4EeuOTBLG4RY78Q,0.618370
1,-3jgpo3XEeuquA5bylYGqQ,0.765102
2,-4iSgbBhEeutEwol7kuJnw,0.791850
3,-D1ayv64Eeuw4w5IkZJKbw,0.393726
4,-DJEc-L5Eeub2BLESLBCkw,0.640788
...,...,...
217916,znnNSmJ0EeySGRI6zc6klQ,0.478050
217917,zoxg94ZkEeup2A5L6jSf_w,0.724342
217918,zqIzpwZmEeyj9BIkvojZuQ,0.224331
217919,zr14itx1EeuYKhKvgjGtpw,0.777335


In [155]:
### PLEASE CHANGE THIS CODE TO IMPLEMENT YOUR OWN PREDICTIONS

# View our 'prediction_df' dataframe as required for submission.
# Ensure it should contain 217,921 rows and 2 columns 'subscription_id' and 'predicted_probaility'
print(prediction_df.shape)
prediction_df.head(10)

(217921, 2)


,subscription_id,predicted_probability
0,-1flsPG4EeuOTBLG4RY78Q,0.618370
1,-3jgpo3XEeuquA5bylYGqQ,0.765102
2,-4iSgbBhEeutEwol7kuJnw,0.791850
3,-D1ayv64Eeuw4w5IkZJKbw,0.393726
4,-DJEc-L5Eeub2BLESLBCkw,0.640788
5,-EbzSDdgEeyiog5l139adw,0.484994
6,-GVVgmMxEeuQZgoplpe76w,0.465515
7,-H4Bil5DEeyGdgqSaUkc7Q,0.031336
8,-O-hHcevEeuPIA4yld1PaQ,0.348223
9,-VFHd3C_EeuyDgqvLkrnfQ,0.813442


**PLEASE CHANGE CODE ABOVE TO IMPLEMENT YOUR OWN PREDICTIONS**

## Final Tests - **IMPORTANT** - the cells below must be run prior to submission

Below are some tests to ensure your submission is in the correct format for grading. Please run the tests below an ensure no assertion errors are thrown.

In [156]:
# FINAL TEST CELLS - please make sure all of your code is above these test cells

# Writing to csv for autograding purposes
prediction_df.to_csv("prediction_submission.csv", index=False)
submission = pd.read_csv("prediction_submission.csv")

assert isinstance(submission, pd.DataFrame), 'You should have a dataframe named prediction_df.'

In [157]:
# FINAL TEST CELLS - please make sure all of your code is above these test cells

assert submission.shape[0] == 217921, 'The dataframe prediction_df should have 217921 rows.'

In [158]:
# FINAL TEST CELLS - please make sure all of your code is above these test cells

assert submission.shape[1] == 2, 'The dataframe prediction_df should have 2 columns.'

In [123]:
# FINAL TEST CELLS - please make sure all of your code is above these test cells

## This cell calculates the auc score and is hidden. Submit Assignment to see AUC score.


## SUBMIT YOUR WORK!

Once we are happy with our `prediction_df` we can now submit for autograding! Submit by using the blue **Submit Assignment** at the top of your notebook. Don't worry if your initial submission isn't perfect as you have multiple submission attempts and will obtain some feedback after each submission!